In [2]:
import pandas as pd
import numpy as np

import duckdb

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import linear_model

### Data Preprocessing

In [3]:
duckdb.sql('INSTALL sqlite; LOAD sqlite;')

In [5]:
#connecting/reading the dataset with duckdb.connect

conn = duckdb.connect('../data/diamonds_train.db',read_only=True)
conn 

In [6]:
#with dbeaver I have analyze the dataset and as there are different tables in the dataset, i am creating individual dataframe with every table.

dcity_df = conn.sql('SELECT* FROM diamonds_city').df()
dcity_df

,city_id,city
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam
1,1e73b1647343b286269d517e6f07e6e07ccef10cd7b785...,Zurich
2,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,Las Vegas
3,5a59ef2e40c5f89adb8c0c6ad0a8019b0e252fff530cf2...,New York City
4,5dd272b4f316b776a7b8e3d0894b37e1e42be3d5d3b204...,Paris
5,68371d5bdaab31b5cbc25fbf94b8f9c1238294fc50e715...,Tel Aviv
6,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,Dubai
7,89c7286890f7347ab235234e74d406596a127ae3679042...,Kimberly
8,ba04d229e496b8383a4df91f5e77c194a95cf1a069b0b2...,Surat
9,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp


In [77]:
#another dataframe with diammonds_clarity table. and so on below....

dclarity_df = conn.sql('SELECT* FROM diamonds_clarity').df()
dclarity_df

,clarity_id,clarity
0,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...,SI2
1,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2
2,66686ae1f0c9c400ba32dc600a34ff0aa173395bcbc2d8...,VVS1
3,7020fd7aaf1656dea5c0c0c3d4bb5a28ebf6243fba95d3...,IF
4,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...,SI1
5,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1
6,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...,VS1
7,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2


In [78]:
dcolor_df = conn.sql('SELECT* FROM diamonds_color').df()
dcolor_df

,color_id,color
0,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,G
1,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,D
2,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,H
3,6da43b944e494e885e69af021f93c6d9331c78aa228084...,J
4,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,I
5,a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0...,E
6,f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d378...,F


In [79]:
dcut_df = conn.sql('SELECT* FROM diamonds_cut').df()
dcut_df

,cut_id,cut
0,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,Very Good
1,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,Ideal
2,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,Good
3,de88c121a82a06352bf1aaceba20578356408a334ba046...,Premium
4,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,Fair


In [80]:
dproperties_df = conn.sql('SELECT* FROM diamonds_properties').df()
dproperties_df

,index_id,cut_id,color_id,clarity_id
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...
...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d378...,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...


In [81]:
dtrans_df = conn.sql('SELECT* FROM diamonds_transactional').df()
dtrans_df

,index_id,price,city_id,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,505,89c7286890f7347ab235234e74d406596a127ae3679042...,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2686,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,738,89c7286890f7347ab235234e74d406596a127ae3679042...,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4882,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02
...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,10070,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,1.34
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,12615,e9c722cbefc2f055ae60b4e2cbe73a2d99537eab0c37f3...,2.02
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,5457,89c7286890f7347ab235234e74d406596a127ae3679042...,1.01
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,456,89c7286890f7347ab235234e74d406596a127ae3679042...,0.33


In [82]:
ddimensions_df = conn.sql('SELECT* FROM diamonds_dimensions').df()
ddimensions_df

,index_id,depth,table,x,y,z
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95
...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76


In [83]:
#now i am merging the dataframes with the common column....

merged1 = pd.merge(dcity_df, dtrans_df, how='inner', on = 'city_id')
merged1

,city_id,city,index_id,price,carat
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,3733,1.01
1,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d6...,1207,0.43
2,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,12932,1.30
3,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,eb1e33e8a81b697b75855af6bfcdbcbf7cbbde9f94962c...,743,0.35
4,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,1da51b8d8ff98f6a48f80ae79fe3ca6c26e1abb7b7d125...,3715,0.90
...,...,...,...,...,...
40450,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,de01c646e02afeb523366bc17518f14454dd06d1da3cc4...,1781,0.51
40451,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,1d2da1e7c1350f25de915e75d5efe8231554c2cc410f6e...,5215,1.36
40452,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,86b7bcae82a5e5eaff69df1e75db3405b385645e27c177...,665,0.32
40453,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,738ca570e9903383f2fb53b684f4b2c46dcdefb8ec303a...,1173,0.57


In [84]:
#and here with the merged dataframe merging a individual dataframe
#trans,city, properties(dataframes those are done merging)

merged2 = pd.merge(merged1, dproperties_df, how='inner', on = 'index_id')
merged2

,city_id,city,index_id,price,carat,cut_id,color_id,clarity_id
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,3733,1.01,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...
1,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d6...,1207,0.43,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...
2,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,12932,1.30,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,7020fd7aaf1656dea5c0c0c3d4bb5a28ebf6243fba95d3...
3,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,eb1e33e8a81b697b75855af6bfcdbcbf7cbbde9f94962c...,743,0.35,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,66686ae1f0c9c400ba32dc600a34ff0aa173395bcbc2d8...
4,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,1da51b8d8ff98f6a48f80ae79fe3ca6c26e1abb7b7d125...,3715,0.90,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0...,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...
...,...,...,...,...,...,...,...,...
40450,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,de01c646e02afeb523366bc17518f14454dd06d1da3cc4...,1781,0.51,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...
40451,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,1d2da1e7c1350f25de915e75d5efe8231554c2cc410f6e...,5215,1.36,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...
40452,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,86b7bcae82a5e5eaff69df1e75db3405b385645e27c177...,665,0.32,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...
40453,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,738ca570e9903383f2fb53b684f4b2c46dcdefb8ec303a...,1173,0.57,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...


In [85]:
#trans,city, properties, clarity

merged3 = pd.merge(merged2, dclarity_df, how='inner', on = 'clarity_id')
merged3

,city_id,city,index_id,price,carat,cut_id,color_id,clarity_id,clarity
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,3733,1.01,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
1,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,d2f483672c0239f6d7dd3c9ecee6deacbcd59185855625...,457,0.34,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
2,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,4a8596a7790b5ca9e067da401c018b3206befbcf95c381...,842,0.32,de88c121a82a06352bf1aaceba20578356408a334ba046...,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
3,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,314f04b30f62e0056bd059354a5536fb2e302107eed143...,845,0.40,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
4,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,a0eaec5a55dc2f5b2ba523018adc485ff620b9d83509b9...,435,0.31,de88c121a82a06352bf1aaceba20578356408a334ba046...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
...,...,...,...,...,...,...,...,...,...
40450,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,d6978e0b553d5157553bdc911b82c4f33c6d6b153dba2e...,5410,1.50,de88c121a82a06352bf1aaceba20578356408a334ba046...,f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d378...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1
40451,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,1acacb59fabe8f0508aa4dac53f015c7ba4728b4083563...,2378,1.05,de88c121a82a06352bf1aaceba20578356408a334ba046...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1
40452,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,e4b3b1d27a50a62fce5e08e00bd7f22a1a9993838303e3...,3011,1.20,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1
40453,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,1821dc44f05e484af1ac4b4d7b6b3ff3bb9f12376d1d13...,7934,2.20,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1


In [86]:
#trans,city, properties, clarity, color

merged4 = pd.merge(merged3, dcolor_df, how='inner', on = 'color_id')
merged4

,city_id,city,index_id,price,carat,cut_id,color_id,clarity_id,clarity,color
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,3733,1.01,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H
1,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,6f90a5a0d3234433d03c7a06fc4bd5c3ac1f21f3397829...,6133,1.04,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H
2,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,7a5df5ffa0dec2228d90b8d0a0f1b0767b748b0a41314c...,2548,0.72,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H
3,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,00037f39cf870a1f49129f9c82d935665d352ffd25ea32...,3963,0.91,de88c121a82a06352bf1aaceba20578356408a334ba046...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H
4,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,d8b5e2791d0d1cee319ee3def0e4631852bfcb329e06fe...,507,0.31,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H
...,...,...,...,...,...,...,...,...,...,...
40450,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,London,e930fc3b53cda9efd0e96418395e4b6748e93b62f7a47c...,5141,1.50,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1,J
40451,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,London,b30f7c7ab21c56f45ab33f28295fc9d6660601c3925a7f...,2774,1.17,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1,J
40452,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,7b31c40281e40541b60824340375dc4d70b2b8da0ec6c7...,5733,2.27,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1,J
40453,ed76f191621cf6ed77eab6a8013d1cb4bdfa89a75a24aa...,Luxembourg,1de9a94dd85dc7680aeeadab8a53c5b16b93a10c1640e4...,8807,2.74,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1,J


In [87]:
#trans,city, properties, clarity, color, cut

merged5 = pd.merge(merged4, dcut_df, how='inner', on = 'cut_id')
merged5

,city_id,city,index_id,price,carat,cut_id,color_id,clarity_id,clarity,color,cut
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,3733,1.01,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair
1,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,aa2942574c9424558eec433914350dbe8c45ce49682e57...,9452,1.51,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair
2,1e73b1647343b286269d517e6f07e6e07ccef10cd7b785...,Zurich,321ef7e85c8c12e11212c6fdeff6249f415294a569d1bb...,12765,2.00,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair
3,1e73b1647343b286269d517e6f07e6e07ccef10cd7b785...,Zurich,3fd60c25b7fc3d4e19f701a606f767d00726fa63bc8ee8...,17713,1.99,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair
4,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,Las Vegas,b24bdc2fb415e6a7038f442fd99a7144f3cfe358086a1b...,2268,0.70,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair
...,...,...,...,...,...,...,...,...,...,...,...
40450,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp,418001574db27512f019b108b1e32d88eba04fcd1fcbb8...,3193,0.90,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2,J,Good
40451,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp,88b27085f448550ed32476d52a91b27f11270eabffaf61...,15898,2.03,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2,J,Good
40452,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp,75e8751a57e0fc4097f6d98b486fe1680635494bbfc7ce...,666,0.38,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2,J,Good
40453,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,Las Vegas,b363886c229e536bd3c84e0c3e89312d70e00422578e07...,1945,0.91,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1,J,Good


In [88]:
#trans,city, properties, clarity, color, cut, dimen

merged6 = pd.merge(merged5, ddimensions_df, how='inner', on = 'index_id')
merged6

,city_id,city,index_id,price,carat,cut_id,color_id,clarity_id,clarity,color,cut,depth,table,x,y,z
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,3733,1.01,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair,65.2,56.0,6.23,6.19,4.05
1,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam,aa2942574c9424558eec433914350dbe8c45ce49682e57...,9452,1.51,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair,65.0,64.0,7.10,7.03,4.59
2,1e73b1647343b286269d517e6f07e6e07ccef10cd7b785...,Zurich,321ef7e85c8c12e11212c6fdeff6249f415294a569d1bb...,12765,2.00,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair,65.2,56.0,7.92,7.87,5.15
3,1e73b1647343b286269d517e6f07e6e07ccef10cd7b785...,Zurich,3fd60c25b7fc3d4e19f701a606f767d00726fa63bc8ee8...,17713,1.99,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair,60.5,63.0,8.20,8.11,4.93
4,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,Las Vegas,b24bdc2fb415e6a7038f442fd99a7144f3cfe358086a1b...,2268,0.70,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,H,Fair,65.0,56.0,5.60,5.50,3.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp,418001574db27512f019b108b1e32d88eba04fcd1fcbb8...,3193,0.90,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2,J,Good,57.4,61.0,6.42,6.37,3.67
40451,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp,88b27085f448550ed32476d52a91b27f11270eabffaf61...,15898,2.03,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2,J,Good,63.2,60.0,7.88,7.94,5.00
40452,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp,75e8751a57e0fc4097f6d98b486fe1680635494bbfc7ce...,666,0.38,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2,J,Good,63.4,56.0,4.56,4.62,2.91
40453,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,Las Vegas,b363886c229e536bd3c84e0c3e89312d70e00422578e07...,1945,0.91,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,c2818bc4e5ec4ae4a357a0df6fed73652e169ec676f7d4...,I1,J,Good,63.1,57.0,6.06,6.12,3.84


In [167]:
#and here is the final dataframe

final_df = merged6[['price', 'carat' , 'depth' , 'table' , 'x' , 'y' , 'z' , 'city', 'cut', 'color', 'clarity']]
final_df

,price,carat,depth,table,x,y,z,city,cut,color,clarity
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,Amsterdam,Fair,H,VS2
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,Amsterdam,Fair,H,VS2
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,Zurich,Fair,H,VS2
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,Zurich,Fair,H,VS2
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,Las Vegas,Fair,H,VS2
...,...,...,...,...,...,...,...,...,...,...,...
40450,3193,0.90,57.4,61.0,6.42,6.37,3.67,Antwerp,Good,J,VVS2
40451,15898,2.03,63.2,60.0,7.88,7.94,5.00,Antwerp,Good,J,VVS2
40452,666,0.38,63.4,56.0,4.56,4.62,2.91,Antwerp,Good,J,VVS2
40453,1945,0.91,63.1,57.0,6.06,6.12,3.84,Las Vegas,Good,J,I1


In [218]:
final_df.to_csv('../data/final_diamonds_train.csv')

In [168]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40455 entries, 0 to 40454
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   price    40455 non-null  int64  
 1   carat    40455 non-null  float64
 2   depth    40455 non-null  float64
 3   table    40455 non-null  float64
 4   x        40455 non-null  float64
 5   y        40455 non-null  float64
 6   z        40455 non-null  float64
 7   city     40455 non-null  object 
 8   cut      40455 non-null  object 
 9   color    40455 non-null  object 
 10  clarity  40455 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 3.7+ MB


In [169]:
# getting the categorical features from the final dataframe
categorical_columns = final_df.select_dtypes(include=['object']).columns
categorical_columns

Index(['city', 'cut', 'color', 'clarity'], dtype='object')

In [170]:
#encoding
final_df = pd.get_dummies(final_df, columns=categorical_columns)
final_df

,price,carat,depth,table,x,y,z,city_Amsterdam,city_Antwerp,city_Dubai,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,3193,0.90,57.4,61.0,6.42,6.37,3.67,0,1,0,...,0,1,0,0,0,0,0,0,0,1
40451,15898,2.03,63.2,60.0,7.88,7.94,5.00,0,1,0,...,0,1,0,0,0,0,0,0,0,1
40452,666,0.38,63.4,56.0,4.56,4.62,2.91,0,1,0,...,0,1,0,0,0,0,0,0,0,1
40453,1945,0.91,63.1,57.0,6.06,6.12,3.84,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [171]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40455 entries, 0 to 40454
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               40455 non-null  int64  
 1   carat               40455 non-null  float64
 2   depth               40455 non-null  float64
 3   table               40455 non-null  float64
 4   x                   40455 non-null  float64
 5   y                   40455 non-null  float64
 6   z                   40455 non-null  float64
 7   city_Amsterdam      40455 non-null  uint8  
 8   city_Antwerp        40455 non-null  uint8  
 9   city_Dubai          40455 non-null  uint8  
 10  city_Kimberly       40455 non-null  uint8  
 11  city_Las Vegas      40455 non-null  uint8  
 12  city_London         40455 non-null  uint8  
 13  city_Luxembourg     40455 non-null  uint8  
 14  city_Madrid         40455 non-null  uint8  
 15  city_New York City  40455 non-null  uint8  
 16  city

In [174]:
final_df.describe()

,price,carat,depth,table,x,y,z,city_Amsterdam,city_Antwerp,city_Dubai,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,...,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,3928.444469,0.797706,61.752841,57.446133,5.729392,5.732819,3.537154,0.061426,0.124756,0.063305,...,0.099666,0.052898,0.013917,0.032802,0.240984,0.171277,0.149944,0.228600,0.068570,0.093907
std,3992.416147,0.475544,1.431725,2.233535,1.124453,1.146650,0.697062,0.240114,0.330446,0.243514,...,0.299558,0.223833,0.117147,0.178120,0.427686,0.376755,0.357021,0.419936,0.252725,0.291703
min,326.000000,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,945.000000,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2397.000000,0.700000,61.800000,57.000000,5.690000,5.710000,3.520000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5331.000000,1.040000,62.500000,59.000000,6.540000,6.540000,4.035000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,18823.000000,4.500000,79.000000,95.000000,10.230000,58.900000,8.060000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Starting Workflow With The Train Set

In [175]:
# creating X and y

X = final_df.drop('price', axis=1)
X.head()

,carat,depth,table,x,y,z,city_Amsterdam,city_Antwerp,city_Dubai,city_Kimberly,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.01,65.2,56.0,6.23,6.19,4.05,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1.51,65.0,64.0,7.10,7.03,4.59,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2.00,65.2,56.0,7.92,7.87,5.15,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1.99,60.5,63.0,8.20,8.11,4.93,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.70,65.0,56.0,5.60,5.50,3.61,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [176]:
y = final_df['price']
y

0         3733
1         9452
2        12765
3        17713
4         2268
         ...  
40450     3193
40451    15898
40452      666
40453     1945
40454     6521
Name: price, Length: 40455, dtype: int64

In [177]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [178]:
# to confirm the sizes of sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (32364, 39)
X_test shape: (8091, 39)
y_train shape: (32364,)
y_test shape: (8091,)


In [179]:
# scaling
scaler = RobustScaler()
scaled_data = scaler.fit_transform(X_train)
scaled_data

array([[ 2.203125  ,  0.26666667,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.359375  , -0.66666667, -0.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [-0.140625  , -1.2       ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.609375  ,  1.13333333, -0.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.46875   , -2.46666667,  2.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.234375  ,  0.33333333,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [180]:
scaled_test = scaler.transform(X_test)
scaled_test

array([[ 0.4375    , -1.26666667,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [-0.21875   , -0.6       , -0.33333333, ...,  0.        ,
         0.        ,  1.        ],
       [-0.21875   ,  0.13333333,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.84375   , -0.26666667,  0.33333333, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.828125  ,  0.73333333,  0.66666667, ...,  1.        ,
         0.        ,  0.        ],
       [-0.671875  , -2.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
# Linear model
'''
linear_model = LogisticRegression(max_iter=1000)
linear_param = linear_model.fit(X_train, y_train)
linear_pred = linear_model.predict(X_test)
linear_auc = roc_auc_score(y_test, linear_pred)
print(f"Linear model AUC is: {linear_auc}")

'''

In [220]:
%%time

# Model definition

#model = LinearRegression()
#model = linear_model.Lasso()
#model = Ridge()
model = ElasticNet()
#model = RandomForestRegressor()

#model = SVR()
#model = SGDRegressor()

hyperparameters = model.get_params()

print(type(model), '\n')
print('Model hyperparameters:', hyperparameters, '\n')

<class 'sklearn.linear_model._coordinate_descent.ElasticNet'> 

Model hyperparameters: {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False} 

CPU times: total: 0 ns
Wall time: 0 ns


In [221]:
%%time

# Model training

model.fit(X_train, y_train)

print('Model:', model, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
print('Model coefficients:', model.coef_, '\n')

Model: ElasticNet() 

Model hyperparameters: {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False} 

Model coefficients: [ 6.58956135e+02 -2.34628979e+01 -1.44230933e+01  1.11259000e+03
  9.63666799e+02  6.59804301e+02  8.62849358e+00 -1.32806270e+01
 -0.00000000e+00  3.36912127e+00 -8.78050063e-01  0.00000000e+00
 -1.87168027e+00  3.86952775e+00 -3.70577987e+00 -1.38502838e+00
  2.94574607e+00 -0.00000000e+00  2.47598992e+00 -5.07800954e+01
 -4.07669765e+01  4.59240177e+01  3.28568104e+01  1.17660780e+01
  3.07285933e+01  1.17010573e+01  2.49116942e+01  6.39863591e+01
 -3.29230104e+01 -2.85035934e+01 -7.09010544e+01 -7.54274737e+01
  5.48361261e+01 -1.20967721e+02 -1.72795738e+02  7.09205913e+01
  5.29066415e+01  7.71827424e+01  1.11344819e+02] 

CPU times: total: 250 ms
Wall time: 149 ms


In [103]:
'''
%%time

# Model predictions

predictions = model.predict(X_test)

print(predictions)
'''

'\n%%time\n\n# Model predictions\n\npredictions = model.predict(X_test)\n\nprint(predictions)\n'

In [104]:

'''
# Visual check

check = pd.DataFrame({'Ground truth':y_test, 'Predictions':predictions, 'Diff':y_test-predictions})
check
'''

"\n# Visual check\n\ncheck = pd.DataFrame({'Ground truth':y_test, 'Predictions':predictions, 'Diff':y_test-predictions})\ncheck\n"

In [105]:
'''
check.reset_index(inplace=True)

check.plot(x='index', y=['Ground truth', 'Predictions'], kind='line', figsize=(10, 3));
'''

"\ncheck.reset_index(inplace=True)\n\ncheck.plot(x='index', y=['Ground truth', 'Predictions'], kind='line', figsize=(10, 3));\n"

In [120]:
'''
# MSE calculation

mean_squared_error(y_test, predictions)
'''

'\n# MSE calculation\n\nmean_squared_error(y_test, predictions)\n'

In [121]:
'''
# RMSE calculation

rmse = mean_squared_error(y_test, predictions, squared=False)
rmse
'''

'\n# RMSE calculation\n\nrmse = mean_squared_error(y_test, predictions, squared=False)\nrmse\n'

In [108]:
'''
# Visual analysis

check = pd.DataFrame({'Ground truth':y_test[:10],
                      'Predictions':predictions[:10], 
                      'RMSE':rmse})
check.reset_index(inplace=True)
check.plot(x='index',
           y=['Ground truth', 'Predictions'], 
           kind='line', 
           figsize=(15, 10),
           yerr=rmse);
'''

"\n# Visual analysis\n\ncheck = pd.DataFrame({'Ground truth':y_test[:10],\n                      'Predictions':predictions[:10], \n                      'RMSE':rmse})\ncheck.reset_index(inplace=True)\ncheck.plot(x='index',\n           y=['Ground truth', 'Predictions'], \n           kind='line', \n           figsize=(15, 10),\n           yerr=rmse);\n"

In [109]:
# training the model on entire dataset
model.fit(X_train, y_train)

LinearRegression()

### Test Set

In [222]:
test_df = pd.read_csv('../data/diamonds_test.csv')
test_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


In [223]:
final_test_df = test_df[['carat' , 'depth' , 'table' , 'x' , 'y' , 'z' , 'city', 'cut', 'color', 'clarity']]

In [224]:
final_test_df.head()

,carat,depth,table,x,y,z,city,cut,color,clarity
0,0.79,62.7,60.0,5.82,5.89,3.67,Amsterdam,Very Good,F,SI1
1,1.20,61.0,57.0,6.81,6.89,4.18,Surat,Ideal,J,VS1
2,1.57,62.2,61.0,7.38,7.32,4.57,Kimberly,Premium,H,SI1
3,0.90,63.8,54.0,6.09,6.13,3.90,Kimberly,Very Good,F,SI1
4,0.50,62.9,58.0,5.05,5.09,3.19,Amsterdam,Very Good,F,VS1


In [225]:
final_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    13485 non-null  float64
 1   depth    13485 non-null  float64
 2   table    13485 non-null  float64
 3   x        13485 non-null  float64
 4   y        13485 non-null  float64
 5   z        13485 non-null  float64
 6   city     13485 non-null  object 
 7   cut      13485 non-null  object 
 8   color    13485 non-null  object 
 9   clarity  13485 non-null  object 
dtypes: float64(6), object(4)
memory usage: 1.0+ MB


In [226]:
final_test_df.describe()

,carat,depth,table,x,y,z
count,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000
mean,0.798642,61.739095,57.490337,5.736454,5.739648,3.543474
std,0.469399,1.435310,2.237109,1.113671,1.128507,0.731005
min,0.200000,50.800000,51.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,4.730000,4.730000,2.920000
50%,0.700000,61.900000,57.000000,5.700000,5.720000,3.530000
75%,1.040000,62.500000,59.000000,6.530000,6.530000,4.040000
max,5.010000,79.000000,73.000000,10.740000,31.800000,31.800000


In [227]:
# getting the categorical features from the dataframe
categorical_cols = final_test_df.select_dtypes(include=['object']).columns
categorical_cols

Index(['city', 'cut', 'color', 'clarity'], dtype='object')

In [228]:
#encoding
final_test_df = pd.get_dummies(final_test_df, columns=categorical_cols)

In [229]:
final_test_df.head()

,carat,depth,table,x,y,z,city_Amsterdam,city_Antwerp,city_Dubai,city_Kimberly,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.79,62.7,60.0,5.82,5.89,3.67,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1.20,61.0,57.0,6.81,6.89,4.18,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,1.57,62.2,61.0,7.38,7.32,4.57,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0.90,63.8,54.0,6.09,6.13,3.90,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,0.50,62.9,58.0,5.05,5.09,3.19,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [242]:
final_test_df.to_csv('../data/final_diamonds_test.csv')

In [230]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       13485 non-null  int64  
 1   carat    13485 non-null  float64
 2   cut      13485 non-null  object 
 3   color    13485 non-null  object 
 4   clarity  13485 non-null  object 
 5   depth    13485 non-null  float64
 6   table    13485 non-null  float64
 7   x        13485 non-null  float64
 8   y        13485 non-null  float64
 9   z        13485 non-null  float64
 10  city     13485 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.1+ MB


In [231]:
test_prediction = model.predict(final_test_df)

In [232]:
test_prediction

array([4105.95824864, 6985.46831803, 8282.75683739, ..., 4018.28068323,
       4012.13676713, 1283.22918533])

In [233]:
# Create a dataframe with the predicted price
predictions_df = pd.DataFrame({'price': test_prediction})
predictions_df

,price
0,4105.958249
1,6985.468318
2,8282.756837
3,4917.347744
4,2186.569086
...,...
13480,2742.062927
13481,3930.221354
13482,4018.280683
13483,4012.136767


,price
id,
0,4105.958249
1,6985.468318
2,8282.756837
3,4917.347744
4,2186.569086
...,...
13480,2742.062927
13481,3930.221354
13482,4018.280683


In [236]:
predictions_df.min()

price   -10135.503939
dtype: float64

In [166]:
predictions_df.to_csv('../data/predict-1.csv')